## Preliminaries

### Packages

In [ ]:
!pip install geopandas

<br>

### Libraries

In [ ]:
import folium
import pandas as pd
import geopandas as gpd

import os
import io
import requests
import zipfile
import folium.plugins
import math


<br>
<br>

## Mathematics

In [ ]:
class Files:

    @staticmethod
    def extractor(blob, path):
        """
        :param blob: A Zip archive's URL
        :param path: The local target directory of the extracts
        """

        try:
            req = requests.get(blob)
        except OSError as err:
            print(err)
            sys.exit(1)

        zipped_object = zipfile.ZipFile(io.BytesIO(req.content))
        zipped_object.extractall(path=path)

    @staticmethod
    def cleanup(path):

        [os.remove(os.path.join(base, file))
            for base, directories, files in os.walk(path)
                for file in files]

        [os.removedirs(os.path.join(base, directory))
            for base, directories, files in os.walk(path, topdown=False)
                for directory in directories
                    if os.path.exists(os.path.join(base, directory))]

    @staticmethod
    def directories(path):
        """
        :param path: The local target directory of the extracts
        """

        if not os.path.exists(path=path):
            os.mkdir(path=path)


<br>
<br>

## Maps

A map wherein the

* centre is latitude = $43.32^{o}N$, longitude = $-71.0589^{o}E$
* the styling is **openstreetmap**; [one of a set](https://github.com/python-visualization/folium/tree/master/folium/templates/tiles)
* **zoom_start** is the initial zoom level, the higher the value, the smaller the area in focus.

In [ ]:

boston = folium.Map(location=[42.32, -71.0589], tiles='openstreetmap', zoom_start=10)
boston

<br>
<br>

### Data

In [ ]:
# Load the data
crimes = pd.read_csv("https://raw.githubusercontent.com/greyhypotheses/hub/develop/data/crime/us/massachusetts/boston/crime.csv", encoding='latin-1')

# Drop rows with missing locations
crimes.dropna(subset=['Lat', 'Long', 'DISTRICT'], inplace=True)

# Focus on major crimes in 2018
crimes = crimes[crimes.YEAR>=2018]
crimes = crimes[crimes.OFFENSE_CODE_GROUP.isin([
    'Larceny', 'Auto Theft', 'Robbery', 'Larceny From Motor Vehicle', 'Residential Burglary',
    'Simple Assault', 'Harassment', 'Ballistics', 'Aggravated Assault', 'Other Burglary',
    'Arson', 'Commercial Burglary', 'HOME INVASION', 'Homicide', 'Criminal Harassment',
    'Manslaughter'])]

In [ ]:
crimes.head()

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location
0,I182070945,619,Larceny,LARCENY ALL OTHERS,D14,808,NaN,2018-09-02 13:00:00,2018,9,Sunday,13,Part One,LINCOLN ST,42.357791,-71.139371,"(42.35779134, -71.13937053)"
6,I182070933,724,Auto Theft,AUTO THEFT,B2,330,NaN,2018-09-03 21:25:00,2018,9,Monday,21,Part One,NORMANDY ST,42.306072,-71.082733,"(42.30607218, -71.08273260)"
8,I182070931,301,Robbery,ROBBERY - STREET,C6,177,NaN,2018-09-03 20:48:00,2018,9,Monday,20,Part One,MASSACHUSETTS AVE,42.331521,-71.070853,"(42.33152148, -71.07085307)"
19,I182070915,614,Larceny From Motor Vehicle,LARCENY THEFT FROM MV - NON-ACCESSORY,B2,181,NaN,2018-09-02 18:00:00,2018,9,Sunday,18,Part One,SHIRLEY ST,42.325695,-71.068168,"(42.32569490, -71.06816778)"
24,I182070908,522,Residential Burglary,BURGLARY - RESIDENTIAL - NO FORCE,B2,911,NaN,2018-09-03 18:38:00,2018,9,Monday,18,Part One,ANNUNCIATION RD,42.335062,-71.093168,"(42.33506218, -71.09316781)"


<br>
<br>

### Daytime Robberies

#### Data

In [ ]:
daytime_robberies = crimes[((crimes.OFFENSE_CODE_GROUP == 'Robbery') & \
                            (crimes.HOUR.isin(range(9,18))))]

<br>
<br>

#### Mapping

In [ ]:
# Create a map
robberies = folium.Map(location=[42.32,-71.0589], tiles='cartodbpositron', zoom_start=13)

# Add points to the map
for idx, row in daytime_robberies.iterrows():
    folium.Marker([row['Lat'], row['Long']]).add_to(robberies)

# Display the map
robberies

<br>
<br>

#### Uncrowded Map

In [ ]:
# Create the map
uncrowded = folium.Map(location=[42.32,-71.0589], tiles='cartodbpositron', zoom_start=13)

# Add points to the map
mc = folium.plugins.MarkerCluster()

for idx, row in daytime_robberies.iterrows():
    if not math.isnan(row['Long']) and not math.isnan(row['Lat']):
        mc.add_child(folium.Marker([row['Lat'], row['Long']]))

uncrowded.add_child(mc)

# Display the map
uncrowded

<br>
<br>

#### Bubble Map & Differentiators

In [ ]:
# Create a base map
byhour = folium.Map(location=[42.32,-71.0589], tiles='cartodbpositron', zoom_start=13)

def color_producer(val):
    if val <= 12:
        return 'forestgreen'
    else:
        return 'darkred'

# Add a bubble map to the base map [The radius can be varied.]
for i in range(0,len(daytime_robberies)):
    folium.Circle(
        location=[daytime_robberies.iloc[i]['Lat'], daytime_robberies.iloc[i]['Long']],
        radius=20,
        color=color_producer(daytime_robberies.iloc[i]['HOUR'])
    ).add_to(byhour)

# Display the map
byhour

<br>
<br>

#### Heatmap

In [ ]:
# Create a base map
density = folium.Map(location=[42.32,-71.0589], tiles='cartodbpositron', zoom_start=12)

# Add a heatmap to the base map
folium.plugins.HeatMap(data=crimes[['Lat', 'Long']].values.tolist(), radius=5).add_to(density)


# Display the map
density

<br>
<br>

#### Chloropleth Map

<br>

Upload Boston Police Districts data

In [ ]:
blob = 'https://github.com/greyhypotheses/hub/raw/develop/data/crime/us/massachusetts/boston/Police_Districts.zip'
path = 'districts'

Files.cleanup(path=path)
Files.directories(path=path)
Files.extractor(blob=blob, path=path)

sections = [directory for _, directories, _ in os.walk(path, topdown=True)
                            for directory in directories]
sections

[]

<br>

Local Directory

In [ ]:
if len(sections) == 0:
    base_directory = ''
else:
    base_directory = sections[0]

<br>

Read-in the data

In [ ]:
districts_full = gpd.read_file(os.path.join(path, base_directory, 'Police_Districts.shp'))
districts_full.head()

,OBJECTID,DISTRICT,ID,BPDGIS_GIS,DISTRICT_,DISTRICT__,ShapeSTAre,ShapeSTLen,geometry
0,1,A15,A15,A,15,15,3.798284e+07,57556.859964,"MULTIPOLYGON (((-71.07416 42.39051, -71.07415 ..."
1,2,A7,A7,A,7,07,1.401151e+08,138468.216080,"MULTIPOLYGON (((-70.99644 42.39557, -70.99644 ..."
2,3,A1,A1,A,1,01,4.445582e+07,49080.786370,"POLYGON ((-71.05200 42.36884, -71.05169 42.368..."
3,4,C6,C6,C,6,06,1.237108e+08,93582.798497,"POLYGON ((-71.04406 42.35403, -71.04412 42.353..."
4,5,D4,D4,D,4,04,7.000553e+07,41040.347429,"POLYGON ((-71.07416 42.35724, -71.07359 42.357..."


In [ ]:
districts = districts_full[["DISTRICT", "geometry"]].set_index("DISTRICT")
districts.head()

,geometry
DISTRICT,
A15,"MULTIPOLYGON (((-71.07416 42.39051, -71.07415 ..."
A7,"MULTIPOLYGON (((-70.99644 42.39557, -70.99644 ..."
A1,"POLYGON ((-71.05200 42.36884, -71.05169 42.368..."
C6,"POLYGON ((-71.04406 42.35403, -71.04412 42.353..."
D4,"POLYGON ((-71.07416 42.35724, -71.07359 42.357..."


<br>

Number of crimes per police district

In [ ]:
plot_dict = crimes.DISTRICT.value_counts()
plot_dict.head()

D4     2885
B2     2231
A1     2130
C11    1899
B3     1421
Name: DISTRICT, dtype: int64

<br>

Hence, the choloropleth map.  Noting that

* The **districts** GeoDataFrame is converted to a GeoJSON FeatureCollection via the **\_\_geo_interface__** attribute
* Argument **data** expects a Pandas Series of values; the values aid the colouring geographical areas.

In [ ]:
# Create a base map
outlines = folium.Map(location=[42.32,-71.0589], tiles='cartodbpositron', zoom_start=12)

# Add a choropleth map to the base map
folium.Choropleth(geo_data=districts.__geo_interface__,
           data=plot_dict,
           key_on="feature.id",
           fill_color='YlGnBu',
           legend_name='Major criminal incidents (Jan-Aug 2018)',
           line_color='gainsboro', fill_opacity=0.4
          ).add_to(outlines)

# Display the map
outlines